In [ ]:
import domojupyter as domo
from datetime import datetime, timezone
import json
import requests
import domojupyter.app_db as app_db
from zipfile import ZipFile
import os




def get_jupyter_account(account_name):

    creds_json = None
    i = 0
    while not creds_json:
        try:
            creds_json = json.loads(domo.get_account_property_value(account_name, 'credentials'))
        except Exception as e:
            print('account error - retrying')
            i = i + 1
            if i > 12 : return
            time.sleep(5)

    return creds_json

# class DomoInstance
#
# create an instance
# takes in an instance_json which must contain the URL for the instance in the key "instance url" in the form of : my-instance.domo.com - no preceeding http or www
# takes in a creds_json which contains a username and password for the instance in the keys "DOMO_USER" and "DOMO_PASSWORD" - or an access_token
class DomoInstance:
    
    # initialize - connect to the instance and get a dev_token
    def __init__(self, instance_json, creds_json):
        self.instance_json = instance_json
        self.headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
        self.username = False
        self.password = False
        self.access_token = False
        self.session_token = False
        self.init_auth(creds_json)
        self.dataset_list = []
        self.code_engine_packages = None
    
    # get_instance_url - return the url of the instance
    def get_instance_url(self):
        return self.instance_json["instance_url"]
    
    def domo_api_request(self, endpoint, request_type, instance=None, auth=None, params=None, headers=None, body=None, raw=False):
        
        url = "https://" + self.get_instance_url() +  endpoint
        
        # debugging
        # print('debugging domo_api_request')
        # print(url)
        
        try:
            if request_type.lower() == 'post' : 
                # print('*** processing post')
                # print('*** headers : ')
                # print(self.headers)
                if raw :response = requests.post(url, json=body, headers=self.headers)
                else : response = json.loads(requests.post(url, json=body, headers=self.headers).text)
            elif request_type.lower() == 'get' : 
                # print('*** processing get')
                # print('*** headers : ')
                # print(self.headers)
                if raw : response = requests.get(url, headers=self.headers, params=params)
                else : response = json.loads(requests.get(url, headers=self.headers, params=params).text)
            
        except requests.exceptions.HTTPError as e:
            print('*** HTTP Error occurred')
            print(str(e))
            return (str(e))
        
        except:
            print('*** Unhandled Error occurred')
            print(str(e))
            return (str(e))
        
        return response
        
    
    # init authentication to the instance
    def init_auth(self, creds_json):
        
        if "DOMO_USERNAME" in creds_json and "DOMO_PASSWORD" in creds_json:
            print("*** Username/Password Auth")
            username = creds_json["DOMO_USERNAME"]
            password = creds_json["DOMO_PASSWORD"]
            self.session_token = self.init_dev_token_user_pass(username, password)
            self.headers['x-domo-authentication'] = self.session_token
            
        elif "accessToken" in creds_json:
            print("*** Access Token Auth")
            self.access_token = creds_json["accessToken"]
            print('*** - 1 - accessToken : ' + creds_json["accessToken"])
            self.headers['x-domo-developer-token'] = self.access_token
        
        elif "access_token" in creds_json:
            print("*** Access Token Auth")
            self.access_token = creds_json["access_token"]
            print('*** - 2 - access_token : ' + creds_json["access_token"])
            self.headers['x-domo-developer-token'] = self.access_token
            
        elif "ACCESS_TOKEN" in creds_json:
            print("*** Access Token Auth")
            self.access_token = creds_json["ACCESS_TOKEN"]
            print('*** - 3 - ACCESS_TOKEN : ' + creds_json["ACCESS_TOKEN"])
            self.headers['x-domo-developer-token'] = self.access_token
        
        else:
            print("*** ERROR - No Auth Type Detected")
            
        return False
    
    # tested - passed
    def init_dev_token_user_pass(self, username, password):
        url = f'https://{self.get_instance_url()}/api/content/v2/authentication'
        payload = {'method': 'password', 'emailAddress': username, 'password': password}
        try:
            response = requests.post(url, json=payload, headers=self.headers)
            dev_token = response.json().get('sessionToken')
            
        except e:
            print('error : ' + str(e))
            dev_token = None

        return dev_token
    
    # populate a list of all of the datasets in this instance
    def populate_datasets(self):
        self.dataset_list = self.retrieve_dataset_list()['dataSources']
        for dataset in self.dataset_list:
            dataset_id = dataset['id']
            dataset_name = dataset['name']
            self.processed_dataset_list.append({"id":dataset_id, "name":dataset_name})
            
        return
    
    def print_datasets(self):
        print(self.processed_dataset_list)
            
            
    
    def process_all_datasets(self):
        # DEBUGGING - don't need all datasets
        gpid_list = None
        self.dataset_list = self.retrieve_dataset_list()['dataSources']
        # print('Partner dataset_list : ' + str(len(self.dataset_list)))
        # print(self.dataset_list)
        for dataset in self.dataset_list:
            self.process_dataset(dataset, gpid_list)
    
    # dataset must be a JSON with id and name
    def process_dataset(self, dataset, gpid_list):
        dataset_id = dataset['id']
        dataset_name = dataset['name']
        # print('processing dataset : ' + dataset_name)
        # print(dataset)
        
        # check for certain datasets to ignore - does this apply everywhere also?
        if 'Media Services Dashboard - Data Dictionary' in dataset_name:
            print('*** - ignoring - ' + dataset_name)
            return
        else:
            # qualify the list of pdp policies to review/work on
            pdp_list = self.generate_pdp_list_to_execute(dataset_id)
            print('returned pdp policy list')
            print(pdp_list)
            for pdp_policy in pdp_list:
                
                pass        
            
    # dataset must be a JSON with id and name
    def get_pdp_policy_from_dataset(self, dataset, policy_name):
        dataset_id = dataset['id']
        dataset_name = dataset['name']
        print('getting policy : ' + policy_name + ' from dataset : ' + dataset_name)
        pdp_list = self.get_pdp_list(dataset_id)
        for pdp_policy in pdp_list:
            if pdp_policy['name'] == policy_name:
                return pdp_policy
        
        
    
    # tested - passed
    def retrieve_dataset_list(self):
        url = f'https://{self.get_instance_url()}/api/data/ui/v3/datasources/search'
        headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'x-domo-authentication': self.api_token
        }
        payload = {
                "entities": ["DATASET"],
                "filters": [],
                "combineResults":"true",
                "query":"*",
                "count":1000,
                "offset":0,
                "sort":{"isRelevance":"false",
                        "fieldSorts":[{"field":"create_date",
                                       "sortOrder":"DESC"}]}}
        response = requests.post(url, headers = headers, json = payload)
        json_response = json.loads(response.text)
        return json_response
    
    # TESTED - PASSED
    def update_pdp_policy(self, datasetId, policyId, pdp_object):
        url = f'https://{self.get_instance_url()}/api/query/v1/data-control/{datasetId}/filter-groups/{policyId}'
        headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'x-domo-authentication': self.api_token
        }
        payload = pdp_object
        response = requests.put(url, json = payload, headers=headers)
        return response
    
    # TESTED - PASSED
    def create_pdp_policy(self, datasetId, pdp_object):
        url = f'https://{self.get_instance_url()}/api/query/v1/data-control/{datasetId}/filter-groups'

        payload = pdp_object
        response = requests.post(url, json = payload, headers = self.headers)
        return response
    
    # TESTED - PASSED
    def delete_pdp_policy(self, datasetId, policyId):
        url = f'https://{self.get_instance_url()}/api/query/v1/data-control/{datasetId}/filter-groups/{policyId}'

        response = requests.delete(url, headers = self.headers)
        return response
        
    # TESTED - PASSED
    def get_pdp_list(self, datasetId):
        url = f'http://{self.get_instance_url()}/api/query/v1/data-control/{datasetId}/filter-groups?options=load_associations,load_filters,include_open_policy'

        response = requests.get(url, headers = self.headers) #, params = {'options':'load_associations,load_filters,include_open_policy'})
        jsonResponse = json.loads(response.text)
        return jsonResponse
    
    # TESTED - PASSED
    def get_pdp_policy(self, datasetId, policy_id):
        url = f'http://{self.get_instance_url()}/api/query/v1/data-control/{datasetId}/filter-groups/{policy_id}'

        response = requests.get(url, headers = self.headers, params = {'options':'load_associations,load_filters,include_open_policy'})
        jsonResponse = json.loads(response.text)
        return jsonResponse
    
    # Get all packages in the instance
    def get_all_packages(self):
        # https://domo-ncv.domo.com/api/codeengine/v2/packages
        endpoint = "/api/codeengine/v2/packages"
        return self.domo_api_request(endpoint, request_type = 'get')

    def get_dataset_id(self, dataset_name):
        dataset = [x for x in self.processed_dataset_list if x["name"]==dataset_name]
        return dataset[0]["id"]
    
    
    def get_package_version(self, package_id, version):
        
        endpoint = f"/api/codeengine/v2/packages/{package_id}/versions/{version}?parts=functions,code"
        
        # DEBUG
        # print(endpoint)
        
        return self.domo_api_request(endpoint, request_type = 'get')
    
        
    
    def monitor_code_engine_code(self, appdb_collection):
        
        self.code_engine_packages = self.get_all_packages()
        
        print('length of code engine packages')
        print(len(self.code_engine_packages))
        # print(self.code_engine_packages)
                
        for package in self.code_engine_packages:
            
            package_id = package["id"]
            package_name = package["name"]
            if "description" in package :
                package_description = package["description"]
            else : package_description = ''
            package_language = package["language"]
            package_environment = package["environment"]
            package_availability = package["availability"]
            package_owner = package["owner"]
            package_versions = package["versions"]
            package_created_on = package["createdOn"]
            package_updated_on = package["updatedOn"]
            package_package_source = package["packageSource"]
            
            # DEBUG
            # print('*** Processing Package : ' + package_name)
            # print('*** Processing ID : ' + package_id)
            
            for version_meta in package_versions:
                
                # print(version_meta)
                version = version_meta["version"]
                version_code_meta = self.get_package_version(package_id, version)
                code = version_code_meta["code"]
                created_by = version_code_meta["createdBy"]
                created_on = version_code_meta["createdOn"]
                updated_by = version_code_meta["updatedBy"]
                updated_on = version_code_meta["updatedOn"]
                
                # Current Schema
                # Code Environment
                # ID
                # Date Captured
                # Code
                # Version
                # Published Date
                # Instance
                
                document = {}
                document["Code Environment"] = "Code Engine"
                document["ID"] = package_id
                document["Name"] = package_name
                document["Date Captured"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                document["Code"] = code
                document["Version"] = version
                document["Published Date"] = updated_on
                document["Instance"] = "domo-ncv.domo.com"
                
                # for debugging
                if package_id == "da729875-7937-4a9c-b780-0465ac36a257" :
                    print("document : ")
                    print(document)
                    
                    # move this out of debugging
                    newDocuments = [{"content":document}] # list of documents
                    response = app_db.create_documents(appdb_collection, newDocuments)
                    # print(response)
                    
    def get_all_custom_apps(self):
        
        endpoint = '/api/apps/v1/designs?checkAdminAuthority=true&deleted=false&direction=desc&parts=owners&parts=creator&parts=thumbnail&search=&withPermission=ADMIN'
        
        return self.domo_api_request(endpoint, request_type = 'get')
    
    
    def get_all_design_versions(self, design_id):
        
        endpoint = f"/domoapps/designs/{design_id}/versions"
        
        return self.domo_api_request(endpoint, request_type = 'get')
    
    
    def get_design_source(self, design_id, design_version):
        
        endpoint = f"/domoapps/designs/{design_id}/versions/{design_version}/assets"
        
        return self.domo_api_request(endpoint, request_type = 'get', raw=True)
    
    def save_response_file(self, response, chunk_size=128):
        
        # file_name = response.headers[]
        file_name = 'temp_source.zip'
        
        with open(f'/home/domo/temp_source/{file_name}', 'wb') as fd:
            for chunk in response.iter_content(chunk_size=chunk_size):
                fd.write(chunk)
                
        return f'/home/domo/temp_source/{file_name}'
        
        
    def unzip_response_file(self, zip_file_path):
        
        try:
            with ZipFile(zip_file_path, 'r') as zip_file:
                zip_file.extractall('/home/domo/temp_source/')
        except:
            return False
            
        return zip_file.namelist()
    
    def write_file_to_AppDB(self, file_path, file_name, custom_app, design_version, appdb_collection):            
        with open(file_path + file_name, 'r') as f:
            code = f.read()

            print(code)
            
        document = {}
        document["Code Environment"] = "Custom Apps"
        document["ID"] = custom_app["id"]
        document["Name"] = custom_app["name"]
        document["Date Captured"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        document["Code"] = code
        document["Version"] = design_version
        document["Published Date"] = custom_app["updatedDate"]
        document["Instance"] = "domo-ncv.domo.com"
        
        newDocuments = [{"content":document}] # list of documents
        response = app_db.create_documents(appdb_collection, newDocuments)

        
    
    def monitor_custom_apps(self, appdb_collection):
        
        self.custom_apps = self.get_all_custom_apps()
        
        for custom_app in self.custom_apps:
            
            app_id = custom_app["id"]
            
            design_versions = self.get_all_design_versions(app_id)
            
            # remove for prod
            if app_id == 'b3c5a044-7ca6-49bb-8f98-ca478c274812' :
                
                print('*** Custom App')
                print(custom_app)
                print('*** Design Versions')
                print(design_versions)
            
                for design_version in design_versions:

                    source = self.get_design_source(app_id, design_version)
                    
                    # print(str(type(source)))
                    # print(source.status_code)
                    # print(source.apparent_encoding)
                    # print(source.headers)
                    zip_file_path = self.save_response_file(source)
                    file_list = self.unzip_response_file(zip_file_path)
                    
                    if file_list:
                        # process files
                        for file_name in file_list:
                            # skip the assets directory, domo.js and thumbnail
                            if 'assets/' not in file_name and file_name != 'thumbnail.png' and file_name != 'domo.js'  : 
                                self.write_file_to_AppDB('/home/domo/temp_source/', file_name, custom_app, design_version, appdb_collection)
                        
                        # remove files
                        for file_name in file_list:
                            
                            print('*** Removing : ' + ('/home/domo/temp_source/' + file_name))
                            os.remove('/home/domo/temp_source/' + file_name)
                            
                        print('*** Removing : ' + '/home/domo/temp_source/temp_source.zip')
                        os.remove('/home/domo/temp_source/temp_source.zip')
        
        print(len(self.custom_apps))
                


